In [3]:
import os
print(os.getcwd())  # 현재 작업 디렉토리 확인


c:\Users\User\Desktop\빅프로젝트\BigProject\visualization


In [5]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString, MultiLineString

# 파일 경로 설정
csv_path = "./testing/광진구_clusters_route 1.csv"
geojson_path = "./testing/road.geojson"

# CSV 데이터 로드
df = pd.read_csv(csv_path)

# 도로 네트워크 로드
gdf_roads = gpd.read_file(geojson_path)

# 도로 네트워크 그래프 생성
G = nx.Graph()

# GeoJSON에서 도로 데이터 추가 (MultiLineString 포함)
for _, row in gdf_roads.iterrows():
    geom = row.geometry
    if isinstance(geom, MultiLineString):
        for line in geom.geoms:
            coords = list(line.coords)
            for i in range(len(coords) - 1):
                G.add_node(coords[i], pos=coords[i])
                G.add_node(coords[i + 1], pos=coords[i + 1])
                G.add_edge(coords[i], coords[i + 1], weight=line.length)
    elif isinstance(geom, LineString):
        coords = list(geom.coords)
        for i in range(len(coords) - 1):
            G.add_node(coords[i], pos=coords[i])
            G.add_node(coords[i + 1], pos=coords[i + 1])
            G.add_edge(coords[i], coords[i + 1], weight=geom.length)

# 노드 개수 확인
if len(G.nodes) == 0:
    raise ValueError("도로 네트워크가 생성되지 않았습니다. road.geojson 데이터를 확인하세요.")

# 0번 클러스터만 필터링
df_cluster_0 = df[df['cluster'] == 0].sort_values(by='order')

# 클러스터 0의 좌표 리스트 (lat, lon 순서)
coordinates = list(zip(df_cluster_0['latitude'], df_cluster_0['longitude']))

# 도로 위의 가장 가까운 노드 찾기
nodes = []
for lat, lon in coordinates:
    closest_node = min(G.nodes, key=lambda node: (node[0] - lon)**2 + (node[1] - lat)**2)
    nodes.append(closest_node)

# 최적 경로 찾기 (도로 네트워크 기반)
route_edges = []
for i in range(len(nodes) - 1):
    path = nx.shortest_path(G, source=nodes[i], target=nodes[i + 1], weight="weight")
    edges = [(path[j], path[j + 1]) for j in range(len(path) - 1)]
    for edge in edges:
        route_edges.append(LineString([edge[0], edge[1]]))

# 경로 데이터를 GeoDataFrame으로 변환
gdf_route = gpd.GeoDataFrame(geometry=route_edges, crs=gdf_roads.crs)

# GeoJSON 파일로 저장
gdf_route.to_file("cluster_0_route.geojson", driver="GeoJSON")

print("GeoJSON 저장 완료: cluster_0_route.geojson")


GeoJSON 저장 완료: cluster_0_route.geojson


In [16]:
import folium
import geopandas as gpd

# 파일 경로 설정
road_geojson = "./testing/road.geojson"
route_geojson = "./cluster_0_route.geojson"

# 지도 중심 좌표 설정 (서울 광진구 기준)
center = [37.54, 127.08]

# 지도 생성
m = folium.Map(location=center, zoom_start=15)

# 최적 경로 GeoJSON 추가 (투명도 적용)
gdf_route = gpd.read_file(route_geojson)
folium.GeoJson(gdf_route, name="Optimized Route", style_function=lambda x: {'color': 'skyblue', 'weight': 3, 'opacity': 0.7}).add_to(m)

# 지도 저장
map_path = "./road_route_map.html"
m.save(map_path)

print(f"HTML 지도 저장 완료: {map_path}")

HTML 지도 저장 완료: ./road_route_map.html


In [14]:
gdf_route.head()

,geometry
0,"LINESTRING (127.09402 37.53794, 127.09378 37.5..."
1,"LINESTRING (127.09378 37.53803, 127.09368 37.5..."
2,"LINESTRING (127.09368 37.53807, 127.09359 37.5..."
3,"LINESTRING (127.09359 37.53792, 127.09341 37.5..."
4,"LINESTRING (127.09341 37.53765, 127.09333 37.5..."


In [7]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from shapely.geometry import LineString, MultiLineString, Point

# 파일 경로 설정
csv_path = "./testing/광진구_clusters_route 1.csv"
geojson_path = "./testing/road.geojson"
output_path = "./testing/cluster_routes.geojson"

# CSV 데이터 로드
df = pd.read_csv(csv_path)

# 도로 네트워크 로드
gdf_roads = gpd.read_file(geojson_path)

# 도로 네트워크 그래프 생성
G = nx.Graph()

# GeoJSON에서 도로 데이터 추가 (MultiLineString 포함)
for _, row in gdf_roads.iterrows():
    geom = row.geometry
    if isinstance(geom, MultiLineString):
        for line in geom.geoms:
            coords = list(line.coords)
            for i in range(len(coords) - 1):
                G.add_edge(coords[i], coords[i + 1], weight=line.length)
    elif isinstance(geom, LineString):
        coords = list(geom.coords)
        for i in range(len(coords) - 1):
            G.add_edge(coords[i], coords[i + 1], weight=geom.length)

# 네트워크 연결성 확인
connected_components = list(nx.connected_components(G))
print(f"네트워크 연결된 구성 요소 개수: {len(connected_components)}")

# 결과 저장할 리스트
data = []

# 모든 클러스터에 대해 반복
gdf_clusters = gpd.GeoDataFrame()
for cluster in df['cluster'].unique():
    df_cluster = df[df['cluster'] == cluster].sort_values(by='order')
    coordinates = list(zip(df_cluster['latitude'], df_cluster['longitude']))
    
    # 도로 위의 가장 가까운 노드 찾기
    nodes = []
    for lat, lon in coordinates:
        closest_node = min(G.nodes, key=lambda node: (node[0] - lon)**2 + (node[1] - lat)**2)
        
        # 노드가 다른 구성 요소에 속해 있는지 확인하고, 연결되지 않으면 강제 연결
        node_component = [c for c in connected_components if closest_node in c]
        if not node_component:
            print(f"연결되지 않은 노드 발견: {closest_node}, 새로운 연결 추가")
            G.add_node((lon, lat))  # 새로운 노드 추가
            G.add_edge((lon, lat), closest_node, weight=1)  # 강제 연결
            closest_node = (lon, lat)
        nodes.append(closest_node)
    
    # 최적 경로 찾기
    route_edges = []
    for i in range(len(nodes) - 1):
        if nx.has_path(G, nodes[i], nodes[i + 1]):  # 경로가 있을 때만 계산
            path = nx.shortest_path(G, source=nodes[i], target=nodes[i + 1], weight="weight")
            edges = [(path[j], path[j + 1]) for j in range(len(path) - 1)]
            for edge in edges:
                route_edges.append(LineString([edge[0], edge[1]]))
        else:
            print(f"경고: {nodes[i]}과 {nodes[i+1]} 사이에 경로가 없음!")
    
    # 데이터 저장
    for idx, row in df_cluster.iterrows():
        data.append({
            "cluster": row['cluster'],
            "order": row['order'],
            "latitude": row['latitude'],
            "longitude": row['longitude'],
            "nearest_node": nodes[idx % len(nodes)],
            "route_geometry": MultiLineString(route_edges),
            "road_geometry": gdf_roads.geometry.iloc[0]
        })

# GeoDataFrame 변환
gdf_result = gpd.GeoDataFrame(data, geometry="route_geometry", crs=gdf_roads.crs)

# GeoJSON 파일로 저장
gdf_result.to_file(output_path, driver="GeoJSON")

print(f"GeoJSON 저장 완료: {output_path}")



네트워크 연결된 구성 요소 개수: 2591
경고: (127.07580030622518, 37.558864016675116)과 (127.07544032899398, 37.56240776911388) 사이에 경로가 없음!
경고: (127.07544032899398, 37.56240776911388)과 (127.07580030622518, 37.558864016675116) 사이에 경로가 없음!
경고: (127.06532136578787, 37.54411685348833)과 (127.06440769824854, 37.54293257595517) 사이에 경로가 없음!
경고: (127.06440769824854, 37.54293257595517)과 (127.06595398365357, 37.54225048351158) 사이에 경로가 없음!
GeoJSON 저장 완료: ./testing/cluster_routes.geojson


In [10]:
gdf_result.head()

,cluster,order,latitude,longitude,nearest_node,route_geometry,road_geometry
0,0,0,37.537581,127.093899,"(127.09402203137506, 37.53793868591606)","MULTILINESTRING ((127.09402 37.53794, 127.0937...",MULTILINESTRING ((127.10043359381774 37.593301...
1,0,1,37.536724,127.095185,"(127.09549965418603, 37.53678846524549)","MULTILINESTRING ((127.09402 37.53794, 127.0937...",MULTILINESTRING ((127.10043359381774 37.593301...
2,0,2,37.533917,127.093586,"(127.09443279444241, 37.5335505096256)","MULTILINESTRING ((127.09402 37.53794, 127.0937...",MULTILINESTRING ((127.10043359381774 37.593301...
3,0,3,37.537737,127.089547,"(127.08944588521187, 37.537723276452816)","MULTILINESTRING ((127.09402 37.53794, 127.0937...",MULTILINESTRING ((127.10043359381774 37.593301...
4,0,4,37.532467,127.089194,"(127.0892558267966, 37.532414414972706)","MULTILINESTRING ((127.09402 37.53794, 127.0937...",MULTILINESTRING ((127.10043359381774 37.593301...


In [27]:
import folium
import geopandas as gpd
import networkx as nx
import random
import pandas as pd

# 파일 경로 설정
route_geojson = "./testing/cluster_routes.geojson"
csv_path = "./testing/광진구_clusters_route 1.csv"
output_map = "./testing/disconnected_routes_map.html"

# GeoJSON 데이터 로드
gdf_routes = gpd.read_file(route_geojson)
df_clusters = pd.read_csv(csv_path)

# 지도 중심 설정 (서울 광진구 기준)
center = [37.54, 127.08]
m = folium.Map(location=center, zoom_start=15)

# 클러스터별 색상 설정
clusters = df_clusters['cluster'].unique()
cluster_colors = {0:'blue',1:'green',2:'purple'}
gdf_routes['color'] = gdf_routes['cluster'].map(cluster_colors)

for _, row in gdf_routes.iterrows():
    color = row['color'] if pd.notna(row['color']) else "#000000"  # NaN이면 기본 검정색 설정
    folium.GeoJson(
        data=row['geometry'].__geo_interface__,
        name=f"Cluster {row['cluster']}",
        style_function=lambda feature, color=color: {'color': color, 'weight': 3}
    ).add_to(m)

# 연결되지 않은 노드 (고립된 노드) 표시
G = nx.Graph()
for _, row in gdf_routes.iterrows():
    G.add_edges_from([(edge.coords[0], edge.coords[1]) for edge in row['geometry'].geoms])

# 고립된 노드 찾기
disconnected_nodes = [node for component in nx.connected_components(G) if len(component) == 1 for node in component]
for node in disconnected_nodes:
    folium.CircleMarker(
        location=[node[1], node[0]],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.8,
        popup="Disconnected Node"
    ).add_to(m)
    
# 모든 클러스터의 쓰레기통 위치 추가
for cluster in df_clusters['cluster'].unique():  
    cluster_data = df_clusters[df_clusters['cluster'] == cluster]
    marker_color = "blue" if cluster == 0 else "green" if cluster == 1 else "purple"  
    
    for _, row in cluster_data.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"Cluster {cluster}, Order: {row['order']}",
            icon=folium.Icon(color=marker_color, icon='trash')
        ).add_to(m)
# 지도 저장
m.save(output_map)

print(f"HTML 지도 저장 완료: {output_map}")


HTML 지도 저장 완료: ./testing/disconnected_routes_map.html


In [25]:
gdf_routes.head()
gdf_routes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   cluster        151 non-null    int32   
 1   order          151 non-null    int32   
 2   latitude       151 non-null    float64 
 3   longitude      151 non-null    float64 
 4   nearest_node   151 non-null    object  
 5   road_geometry  151 non-null    object  
 6   geometry       151 non-null    geometry
 7   color          151 non-null    object  
dtypes: float64(2), geometry(1), int32(2), object(3)
memory usage: 8.4+ KB
